This script runs one param_set. Outputs results{}.csv and trialxtrial{}.csv

In [1]:
import sys

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
from sklearn import metrics
import pandas as pd

from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from scipy.special import logsumexp
from scipy.stats import norm
from glob import glob
from CSWSEM import generate_exp, seed_exp
from vanilla_lstm import VanillaLSTM
from sem.event_models import NonLinearEvent
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

# used in the experimental design
from sklearn.metrics import adjusted_rand_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing # used sparingly
from scipy.special import logsumexp

# SEM
from sem import sem_run_with_boundaries, SEM
from sem.hrr import plate_formula, encode, decode, embed_gaussian
from sem.utils import fast_mvnorm_diagonal_logprob, get_prior_scale
from no_split_sem import no_split_sem_run_with_boundaries, NoSplitSEM


TensorFlow Version: 2.3.1


### main

In [2]:
from CSWSEM import *
embed_dim = n_verbs = 10
n_train,n_test = 40,10
get_embedding_library(embed_dim,n_verbs,n_train,n_test)

{'Verb0': array([[-0.03650457,  0.41594857, -0.41348452,  0.42319356,  0.12067676,
         -0.35633525, -0.17153576,  0.36257666, -0.04570738,  0.22835637]]),
 'Verb1': array([[-0.0189807 ,  0.68966802,  0.14076405,  0.51183245,  0.38938015,
          0.01312059,  0.29229518,  0.12826641,  0.11902625, -0.09860795]]),
 'Verb2': array([[ 0.06712214,  0.77054304,  0.37369213,  0.3799089 ,  0.18734978,
          0.0604352 , -0.18048568,  0.5835324 , -0.18176583, -0.10611857]]),
 'Verb3': array([[ 0.35617041,  0.20967945,  0.17219219,  0.78820583,  0.13646942,
         -0.24938187, -0.04485196,  0.20772196,  0.23945612,  0.39546008]]),
 'Verb4': array([[ 0.16225036,  0.32134121,  0.13969043,  0.73330326,  0.21652689,
          0.27619072,  0.07939499, -0.05565318, -0.215815  ,  0.11471161]]),
 'Verb5': array([[1.23086017e-01, 5.91496097e-01, 1.39818366e-01, 5.69639226e-01,
         1.82933663e-01, 2.55471972e-01, 1.31387144e-01, 2.77492372e-01,
         3.03330939e-04, 3.67810999e-01]]),
 

### check: run SEM 

In [9]:
# parameter search over lr, n_epochs, alpha, lambda
model_type        = str('LSTM')
lr                = float(0.005)
n_epochs          = int(16)    
log_alpha         = float(0.0)  # sCRP alpha is set in log scale
log_lambda        = float(0.0)  # sCRP lambda is set in log scale
seed = int(99)
condition = 'single'
# number of trials
n_train = 40
n_test = 10

# optimizer
optimizer_kwargs = dict(
    lr=lr, 
    beta_1=0.9, 
    beta_2=0.999, 
    epsilon=1e-5, 
    amsgrad=False
) 

f_opts=dict(
  batch_size=25, 
  batch_update=False, 
  dropout=0.0,
  l2_regularization=0.0, 
  n_epochs=n_epochs,
  optimizer_kwargs=optimizer_kwargs
)


# final param dict
sem_kwargs = dict(
  lmda=np.exp(log_lambda), 
  alfa=np.exp(log_alpha), 
  f_opts=f_opts, 
  f_class=VanillaLSTM
)


In [15]:

x, y, e, _ = generate_exp(condition, n_train=n_train, n_test=n_test, embedding_library=None)

run_kwargs = dict(save_x_hat=True, progress_bar=False)
_sem_results = no_split_sem_run_with_boundaries(x, sem_kwargs, run_kwargs)

In [16]:
_sem_results

# Run model

main fun call

In [6]:
""" 
main fun call
"""

results, trialXtrial, _ = seed_exp( 
              sem_kwargs, model_type=model_type, 
              n_train=n_train, n_test=n_test,
              condition=condition,seed=seed,
)

seed 97 condition single
seed in exp 97


//anaconda3/envs/sem/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [7]:
# convert from JSON file format (dict) to pandas df
results = pd.DataFrame(results)
trialXtrial = pd.DataFrame(trialXtrial)

# save


In [10]:
results_fpath = save_dir + "results_" + model_tag + '.csv'
trial_fpath = save_dir + "trial_X_trial_" + model_tag + '.csv'

LSTM_cond_single_lr_0.010_nepchs_1_alpha_0.000-lambda_0.000_seed_97


In [11]:
results.to_csv(results_fpath)
trialXtrial.to_csv(trial_fpath)